In [193]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

data1 = pd.read_csv('fraud_data.csv').head(4000)
data1



,Profession,Income,Credit_card_number,Expiry,Security_code,Fraud
0,DOCTOR,42509,3515418493460774,07/25,251,1
1,DOCTOR,80334,213134223583196,05/32,858,1
2,LAWYER,91552,4869615013764888,03/30,755,1
3,LAWYER,43623,341063356109385,01/29,160,1
4,DOCTOR,22962,4707418777543978402,11/30,102,0
...,...,...,...,...,...,...
3995,LAWYER,29065,4172253280343459,12/32,45,1
3996,LAWYER,97237,375305958401093,12/25,7973,1
3997,LAWYER,11145,6568376187479279,09/30,804,0
3998,DOCTOR,39396,4618252776559,08/29,121,0


In [194]:
def tidy(data):

# Apply the function to the Expiry column
    data[['Month_Expiry', 'Year_Expiry']] = data['Expiry'].apply(lambda x: pd.Series(refactor_date(x)))

# Apply the Profession transformation
    #data["Profession"] = data["Profession"].apply(lambda x: 1 if x == "DOCTOR" else 2 if x == "LAWYER" else 0)

# Drop the Expiry column
    data.drop("Expiry", axis=1, inplace=True)

# Display the updated DataFrame
# Function to refactor the date
def refactor_date(x):
    x = str(x).strip()  # Strip any extra spaces
    month, year = x.split("/")  # Split the date into month and year
    return int(month), int(year)

tidy(data1)




KeyError: "['Expiry'] not found in axis"

In [223]:
not_fraud = data1[data1.Fraud == 0]
fraud = data1[data1.Fraud == 1]

not_fraud_sample = not_fraud.sample(n=500) #takes a random sample of 492 rows from the not_fraud DataFrame.
new_dataset = pd.concat([not_fraud_sample, fraud], axis=0)

In [186]:
new_dataset['Fraud'].value_counts()

Fraud
1    2007
0     492
Name: count, dtype: int64

In [198]:
data1

,Profession,Income,Credit_card_number,Security_code,Fraud,Month_Expiry,Year_Expiry
0,DOCTOR,42509,3515418493460774,251,1,7,25
1,DOCTOR,80334,213134223583196,858,1,5,32
2,LAWYER,91552,4869615013764888,755,1,3,30
3,LAWYER,43623,341063356109385,160,1,1,29
4,DOCTOR,22962,4707418777543978402,102,0,11,30
...,...,...,...,...,...,...,...
3995,LAWYER,29065,4172253280343459,45,1,12,32
3996,LAWYER,97237,375305958401093,7973,1,12,25
3997,LAWYER,11145,6568376187479279,804,0,9,30
3998,DOCTOR,39396,4618252776559,121,0,8,29


In [218]:

X = new_dataset.drop(columns='Fraud', axis=1)
Y= new_dataset["Fraud"]
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)
categorical_cols = ['Profession']  # Replace with actual categorical column names
numerical_cols = ['Income', 'Security_code','Month_Expiry','Year_Expiry']
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),  # Standardize numerical columns
        ('cat', OrdinalEncoder(), categorical_cols)  # Apply ordinal encoding to categorical columns
    ]
)

# Create a pipeline with SMOTE, preprocessing, and Logistic Regression
model = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),  # Apply SMOTE to balance the classes
    ('classifier', LogisticRegression(random_state=42))  # Logistic Regression model
])

model.fit(X_Train, Y_Train)



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Income', 'Security_code',
                                                   'Month_Expiry',
                                                   'Year_Expiry']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['Profession'])])),
                ('smote', SMOTE(random_state=42)),
                ('classifier', LogisticRegression(random_state=42))])

In [219]:
from sklearn.metrics import accuracy_score

predictions = model.predict(X_Train)

training_data_accuracy = accuracy_score(predictions, Y_Train)
print('Accuracy on Training data : ', training_data_accuracy)

predictions

Accuracy on Training data :  0.548936170212766


array([1, 0, 0, ..., 0, 0, 0])

In [220]:
from sklearn.metrics import accuracy_score

predictions = model.predict(X_Test)

training_data_accuracy = accuracy_score(predictions, Y_Test)
print('Accuracy on Training data : ', training_data_accuracy)


Accuracy on Training data :  0.5558252427184466


In [221]:
data2 = pd.read_csv('fraud_data.csv').tail(2000)
tidy(data2)
X = data2.drop(columns='Fraud' , axis = 1)
Y = data2['Fraud']

predictions = model.predict(X)
training_data_accuracy = accuracy_score(predictions, Y)
training_data_accuracy


0.4975

TypeError: LinearClassifierMixin.predict() takes 2 positional arguments but 3 were given